### Nota importante!

Este notebook funcionou com uma versão 12.2 LTS Runtime!

Não há preocupações quanto ao tamanho do cluster (memória e núcleos).

Insira `spark.databricks.delta.retentionDurationCheck.enabled false` durante a criação das configurações do cluster para poder usar o VACUUM corretamente neste notebook.

O objetivo deste notebook é apenas mostrar a versão dos comandos SQL para Python, **use sempre a versão SQL como referência**, pois foi a usada durante o curso SQL do Databricks.

## Usando DBFS

In [0]:
%fs ls /

In [0]:
display(dbutils.fs.ls('/'))

## Alguns exemplos de comandos DBFS semelhantes a Unix

| Comando | Resultado |
|--------|--------|
| `ls` | Liste os arquivos do diretório. |
| `mkdirs` | Crie um novo diretório. |
| `cp` | Capaz de copiar arquivos de um diretório para outro. |
| `mv` | Capaz de mover arquivos de um diretório para outro. |
| `rm` | Pode remover um arquivo de um caminho. |
| `rm -r` | Remove recursivamente pastas ou arquivos aninhados. |
| `put` | Permite inserir dados ou até mesmo criar novos arquivos de dados. |
| `head` | Permite a leitura de arquivos nas primeiras linhas. |
| `mount` | Crie um link entre um espaço de trabalho e o armazenamento de objetos em nuvem. |
| `secrets.get` | Colete um segredo de um determinado escopo de segredos do Databricks. |


In [0]:
%fs help

In [0]:
help(dbutils.fs)

In [0]:
%fs mkdirs /databricks_cc_example

In [0]:
%fs ls /databricks_cc_example/

In [0]:
dbutils.fs.put("/databricks_cc_example/data.csv", """Name, Age, Job
Angelo, 25, Nurse
Maria, 23, Architect
Ronaldo, 33, Actor
Jessica, 18, Student
Mara, 27, Doctor""",
True)

In [0]:
%fs ls /databricks_cc_example

In [0]:
%fs head /databricks_cc_example/data.csv

In [0]:
%fs ls /

In [0]:
%fs 
ls /mnt/

In [0]:
%fs mounts

In [0]:
access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key")
encoded_secret_key = secret_key.replace("/", "%2F")
aws_bucket_name = "databricks-cc-eng-academy"
mount_name = "s3_dbfs"

dbutils.fs.mount(f"s3a://{access_key}:{encoded_secret_key}@{aws_bucket_name}", f"/mnt/{mount_name}")

In [0]:
top_secret = dbutils.secrets.get(scope = "databricks", key = "secrets")
for letter in top_secret:
    print(letter)

In [0]:
%fs mounts

In [0]:
%fs ls /mnt/s3_dbfs/

In [0]:
spark.sql(
"CREATE TABLE IF NOT EXISTS remuneracao_day2" + \
"USING csv" + \
"OPTIONS (header 'true'," + \
"      delimiter ';'," + \
"      path '/mnt/s3_dbfs/remuneracao202206.csv')"
)

In [0]:
display(df_remuneracao_day2.history())

In [0]:
display(df_remuneracao_day2.describe())

In [0]:
df_remuneracao_day2.createOrReplaceTemporaryView("remuneracao_day2")
display(spark.sql("DESCRIBE TABLE EXTENDED remuneracao_day2"))

In [0]:
#dbutils.fs.unmount(/mnt/s3_dbfs)
#%fs unmount /mnt/s3_dbfs

In [0]:
display(df_remuneracao_day2.limit(5))

In [0]:
#FONTE DO DICIONÁRIO DE DADOS: http://dados.df.gov.br/dataset/462126f8-8a61-4cec-91a2-38615b7f70f6/resource/0514402e-cd29-440b-83e5-f97787dd1ad3/download/dicdadosremuneracaodosservidores.html
spark.sql(
"CREATE TABLE IF NOT EXISTS remuneracao_day (nome STRING COMMENT 'NOME COMPLETO DO SERVIDOR', " + \
"                                            cpf STRING COMMENT 'Nº DO CADASTRO DE PESSOA FÍSICA MASCARADO'," + \
"                                            orgao STRING COMMENT 'ÓRGÃO VINCULADO AO SERVIDOR'," + \
"                                            cargo STRING COMMENT 'POSIÇÃO QUE O SERVIDOR OCUPA NO ÓRGÃO DE FORMA PERMANENTE'," + \
"                                            funcao STRING COMMENT 'DESIGNAÇÃO TEMPORÁRIA PARA DESEMPENHO DE DETERMINADAS ATRIBUIÇÕES (CARGO COMISSIONADO)'," + \
"                                            situacao STRING COMMENT 'CONDIÇÃO DO SERVIDOR EM RELAÇÃO AO EXERCÍCIO DE SUAS ATIVIDADES'," + \
"                                            mes INTEGER COMMENT 'MÊS DE REFERÊNCIA'," + \
"                                            ano INTEGER COMMENT 'ANO DE REFERÊNCIA'," + \
"                                            codigo_do_orgao INTEGER COMMENT 'ÓRGÃO VINCULADO AO SERVIDOR'," + \
"                                            matricula STRING COMMENT 'MATRÍCULA DO SERVIDOR NO ÓRGÃO VINCULADO'," + \
"                                            remuneracao_basica STRING COMMENT 'REMUNERAÇÃO BÁSICA'," + \
"                                            beneficios STRING COMMENT 'VALOR DOS BENEFÍCIOS'," + \
"                                            valor_das_funcoes STRING COMMENT 'VALOR DAS FUNÇÕES'," + \
"                                            comissao_conselheiro STRING COMMENT 'COMISSÃO CONSELHEIRO'," + \
"                                            hora_extra STRING COMMENT 'TOTAL DE HORA EXTRA'," + \
"                                            verbas_eventuais STRING COMMENT 'VALOR DAS VERBAS EVENTUAIS'," + \
"                                            verbas_judiciais STRING COMMENT 'VALOR DAS VERBAS JUDICIAIS'," + \
"                                            descontos_a_maior STRING COMMENT 'VALOR DA REPOSIÇÃO DE DESCONTOS A MAIOR'," + \
"                                            licenca_premio STRING COMMENT 'VALOR DA LICENÇA PRÊMIO'," + \
"                                            irrf STRING COMMENT 'VALOR DO DESCONTO DE IMPOSTO DE RENDA RETIDO NA FONTE'," + \
"                                            seguridade_social STRING COMMENT 'VALOR DO DESCONTO DE SEGURIDADE SOCIAL'," + \
"                                            teto_redutor STRING COMMENT 'VALOR DO DESCONTO DE TETO REDUTOR'," + \
"                                            outros_recebimentos STRING COMMENT 'VALOR DE OUTROS RECEBIMENTOS'," + \
"                                            outros_descontos_obrigatorios STRING COMMENT 'VALOR DE OUTROS DESCONTOS OBRIGATÓRIOS'," + \
"                                            pagamento_a_maior STRING COMMENT 'VALOR DOS DESCONTOS DE PAGAMENTOS A MAIOR'," + \
"                                            bruto STRING COMMENT 'VALOR BRUTO DA REMUNERAÇÃO'," + \
"                                            liquido STRING COMMENT 'VALOR LÍQUIDO APÓS DESCONTOS OBRIGATÓRIOS')" + \
"COMMENT 'Este conjunto de dados apresenta a remuneração dos servidores do Governo do DF, detalhada por órgão e nome do servidor.'"
)

In [0]:
df_remuneracao_day2.selectExpr("*").write.insertInto("remuneracao_day")

## Unity Catalog

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS hive_to_uc_sync")
spark.sql("USE CATALOG hive_to_uc_sync")
spark.sql("CREATE SCHEMA IF NOT EXISTS sync_data")
spark.sql("USE SCHEMA sync_data")

In [0]:
spark.sql("SYNC SCHEMA hive_to_uc_sync.sync_data FROM hive_metastore.default DRY RUN")

### CLONANDO TABELAS


| Deep Clone Features | Shallow Clone Features |
|--------|--------|
| Completely independent of the source. | It depends exclusively on the source. |
| Duplicated all data & metadata | Duplicates metadata only. |
| You can clone a Deep Clone Table | Cannot create Shallow Clone from another Shallow Clone Table. |
| Works with Managed and Unmanaged tables | Works only with Managed Tables on Unity Catalog. |
| Scenarios: migrations, back-ups, upgrade Hive -> UC  | Scenarios: tests, short-duration workloads, low-cost experiments. |

PS: Not only exclusive for Delta Tables as you also can clone Parquet and Iceberg tables.

In [0]:
df_people = spark.read.table("default.people_10millions")

In [0]:
df_people.clone("dbfs://clone_path/deep_clone", isShallow = False, replace)

In [0]:
df_people.clone("dbfs://clone_path/shallow_clone", isShallow, replace)

In [0]:
display(df_people.count())

In [0]:
df_people_deep = spark.read.load("dbfs://clone_path/deep_clone")
display(df_people_deep.count())

In [0]:
df_people_shallow = spark.read.load("dbfs://clone_path/shallow_clone")
display(df_people_shallow.count())

In [0]:
display(df_people_deep.history())

In [0]:
df_people.delete("birthDate >= '2000-01-01'")

In [0]:
display(df_people.count())

In [0]:
display(df_people_deep.count())

In [0]:
display(df_people_deep.history())

In [0]:
df_people.restoreToVersion(0)

In [0]:
display(df_people.count())

#### Para habilitar uma duração fora do mínimo permitido (168h / 1 semana) você pode definir:
`spark.databricks.delta.retentionDurationCheck.enabled falso`
como configurações de parâmetros em seu cluster.

In [0]:
df_people.vacuum(0)

In [0]:
df_people.clone("dbfs://hive_to_uc_sync/people_deep_clone", isShallow = False, replace)

In [0]:
df_titanic = spark.read.table("default.titanic_clean")
df_titanic.clone("dbfs://hive_to_uc_sync/titanic_clean", isShallow = False, replace)

In [0]:
df_titanic = spark.read.table("default.remuneracao_day")
df_remuneracao_day.clone("dbfs://hive_to_uc_sync/remuneracao_day", isShallow = False, replace)

## Lidando com visualizações dinâmicas

In [0]:
table_path = "dbfs://hive_to_uc_sync/remuneracao_day"
df_remuneracao_day = spark.read.load(table_path)
display(df_remuneracao_day.limit(5))

In [0]:
df_remuneracao_day.createOrReplaceTemporaryView("remuneracao_day")
display(spark.sql(
"SELECT" + \
"      mask(cpf,"O","i","*") AS cpf_masked, c" + \
"      cargo, " + \
"      ano, " + \
"      bruto" + \
"FROM" + \
"  remuneracao_day" + \
"LIMIT 10"
))

## Criptografia simétrica

O padrão de criptografia avançado (AES) é uma especificação usada como chaves criptográficas simétricas sincronais, assumindo 128 bytes, 192 bytes ou 256 bytes de tamanho:

| Tamanho do byte AES | Chave de comprimento |
|--------|--------|
| AES-128 | 16 |
| AES-192 | 24 |
| AES-256 | 32 |

In [0]:
display(spark.sql("SELECT aes_encrypt('Day 2 - Fighter - Databricks SQL: From Zero to Hero', 'databricks012345databricks012345')"))

In [0]:
display(spark.sql("SELECT CAST(aes_decrypt(unbase64('KAwERKYuitmalltqEsPLuC7YcLH3kNNMU45J4gjULPr3ijL7Wt9K3X5vpR7uPDtjoo0/dUISR2+qY5HvSom/NHgmtuBQHQds2PKYyiYirw=='), 'databricks012345databricks012345') AS STRING)"))

In [0]:
display(spark.sql(
"SELECT" + \
"      aes_encrypt(cpf , 'databricks012345databricks012345') AS encrypted_cpf, " + \
"      cargo, " + \
"      ano, " + \
"      bruto" + \
"FROM" + \
"  remuneracao_day" + \
"LIMIT 10"
))

In [0]:
spark.sql(
"CREATE VIEW remuneracao_column_security AS " + \
"SELECT" + \
"    CASE" + \
"      WHEN is_account_group_member('ML_Team') THEN 'ACESSO NEGADO'" + \
"      ELSE cpf" + \
"    END AS cpf," + \
"    cargo," + \
"    ano," + \
"    bruto" + \
"FROM" + \
"  remuneracao_day"
)

In [0]:
display(spark.sql("SELECT * FROM remuneracao_column_security LIMIT 5"))

In [0]:
display(spark.sql(
"SELECT + \
"    cargo," + \
"    COUNT(cpf) AS TOTAL" + \
"FROM" + \
"    remuneracao_day" + \
"GROUP BY CARGO" + \
"ORDER BY 2 DESC"
))

In [0]:
spark.sql(
"CREATE VIEW remuneracao_row_column_security AS " + \
"SELECT" + \
"    CASE" + \
"      WHEN is_account_group_member("ML_Team") THEN "ACCESSO NEGADO"" + \
"      ELSE cpf" + \
"    END AS cpf," + \
"    cargo," + \
"    ano," + \
"    bruto" + \
"FROM" + \
"    remuneracao_day" + \
"WHERE" + \
"    CASE" + \
"      WHEN is_account_group_member('ML_Team') THEN CARGO NOT LIKE '%TECNICO ENFERMAGEM%'" + \
"      ELSE TRUE" + \
"    END"
)

In [0]:
display(spark.sql(
"SELECT" + \
"    cargo," + \
"    COUNT(cpf) AS TOTAL" + \
"FROM" + \
"    hive_to_uc_sync.sync_data.remuneracao_row_column_security" + \
"GROUP BY cargo" + \
"ORDER BY 2 DESC"
))

## VOLUMES

In [0]:
spark.read.option("header", True).format("csv").load('/Volumes/titanic/clean/volume/titanic.csv').show()

In [0]:
#df_clone_covid_shallow = df_covid_clone.clone('dbfs://hive_to_uc_sync/deep_clone', isShallow = True)
df_remuneracao_volume = spark.read.load('dbfs://hive_to_uc_sync/remuneracao_deep_clone')
df_remuneracao_volume.write.saveAsTable("hive_to_uc_sync.sync_data.remuneracao_volume")

In [0]:
#df_remuneracao_volume.write.mode("overwrite").option("truncate", "true").save()
spark.sql("TRUNCATE TABLE hive_to_uc_sync.sync_data.remuneracao_volume")

In [0]:
spark.sql(
"COPY INTO remuneracao_volume" + \
"FROM '/Volumes/hive_to_uc_sync/sync_data/volume'" + \
"FILEFORMAT = CSV" + \
"FORMAT_OPTIONS ('mergeSchema' = 'true'," + \
"                  'inferSchema' = 'true'," + \
"                  'delimiter' = ';'," + \
"                  'encoding' = 'ISO-8859-1'," + \
"                  'header' = 'true')" + \
"COPY_OPTIONS ('mergeSchema' = 'true')"
)

In [0]:
display(spark.sql("SELECT COUNT(*) FROM remuneracao_volume"))

In [0]:
display(spark.sql("SELECT * FROM remuneracao_volume LIMIT 5"))